In [23]:

#%matplotlib inline

import glob
import os, time
import itertools
import numpy as np
import seaborn as sns
import pandas as pd
from natsort import natsorted

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cycler import cycler
from datetime import datetime, timedelta

from scipy.stats import norm
from scipy.optimize import curve_fit #we could import more, but this is what we need
from scipy.stats import linregress

from scipy import sparse
from scipy.sparse.linalg import spsolve
from pybaselines import Baseline, utils

from scipy.fft import fft, rfft
from scipy.fft import fftfreq, rfftfreq
from scipy import fftpack
from scipy.fftpack import fft, ifft

%matplotlib inline

In [24]:
'''
Get paths to corresponsing folders
'''

# get the path to the project's main directory (e.g., Project051)
BASE_DIR_PATH = os.path.dirname(os.path.abspath(os.getcwd()))

# ------------------------- data folder -----------------------

# get the path to the folder containing raw data
DATA_DIR_PATH = os.path.join(BASE_DIR_PATH, 'data','')

# get the path to the folder containing UV-Vis-NIR raw data
DATA_UV_DIR_PATH_AGGREGATION = os.path.join(BASE_DIR_PATH, 'data', 'UV-Vis-NIR','2024-01-17-100ns-20nr-25nacl-100h2o-7bp2','15c-2a','')
DATA_UV_DIR_PATH_FT = os.path.join(BASE_DIR_PATH, 'data', 'UV-Vis-NIR','2024-01-17-100ns-20nr-25nacl-100h2o-7bp2','')

# The path to plot spectra taken at different powers

DATA_UV_DIR_PATH_AGGREGATION_1_8 = os.path.join(BASE_DIR_PATH, 'data', 'UV-Vis-NIR','2024-01-24-100ns-20nr-25nacl-100h2o-7bp2','15c-1-8a','')
DATA_UV_DIR_PATH_AGGREGATION_2_0 = os.path.join(BASE_DIR_PATH, 'data', 'UV-Vis-NIR','2024-01-17-100ns-20nr-25nacl-100h2o-7bp2','15c-2a','')
DATA_UV_DIR_PATH_AGGREGATION_2_2 = os.path.join(BASE_DIR_PATH, 'data', 'UV-Vis-NIR','2024-01-24-100ns-20nr-25nacl-100h2o-7bp2','15c-2-2a','')
DATA_UV_DIR_PATH_AGGREGATION_2_5 = os.path.join(BASE_DIR_PATH, 'data', 'UV-Vis-NIR','2024-01-19-100ns-20nr-25nacl-100h2o-7bp2','15c-2-5a','')

# ------------------------- output folder -----------------------

# # get the path to the folder with generated figures
RESULT_DIR_PATH = os.path.join(BASE_DIR_PATH, 'results', 'figs', '')
RESULT_DIR_PATH_SI = os.path.join(BASE_DIR_PATH, 'results', 'figs', 'si_figs', 'png', '')



# For different laser powers

In [ ]:
'''
Import cooling data from UV-Vis-NIR folder
'''

# Get a list of all CSV files in the folder
txt_files_1_8 = glob.glob(os.path.join(DATA_UV_DIR_PATH_AGGREGATION_1_8, '*.txt'))
txt_files_2_0 = glob.glob(os.path.join(DATA_UV_DIR_PATH_AGGREGATION_2_0, '*.txt'))
txt_files_2_2 = glob.glob(os.path.join(DATA_UV_DIR_PATH_AGGREGATION_2_2, '*.txt'))
txt_files_2_5 = glob.glob(os.path.join(DATA_UV_DIR_PATH_AGGREGATION_2_5, '*.txt'))


#================ plot data heating cycle ===========================#

#fig = plt.figure(figsize=(4,2), dpi=300) # generate a figure
#ax = plt.subplot(1,1,1) #(nrow, ncolumn, index) - make a subplot, there is only one

# Initialize the maximum value and its corresponding index
max_values_cool_1_8 = []
max_indexes_cool_1_8 = []
wv_abs_cool_1_8 = []
times_cool_1_8 = []

time_cool = 0
for i, txt_file in enumerate(natsorted(txt_files_1_8)):
    # Read the CSV file into a Pandas DataFrame
    df_cool = pd.read_csv(txt_file, skiprows=14, delimiter='\t', header=None)
    new_df_cool = df_cool.iloc[400:1200, ] #385 to 743 nm
    
    # Find the maximum value and its corresponding index
    #max_value_cool = new_df_cool.iloc[667,1] # 685 nm
    max_value_cool = new_df_cool[1].max()
    max_values_cool_1_8.append(max_value_cool)
    
    # Get the corresponding value from the first column
    max_index_cool = new_df_cool.loc[new_df_cool[1].idxmax(), 0]
    max_indexes_cool_1_8.append(max_index_cool)
    
    times_cool_1_8.append(time_cool)
    time_cool = time_cool + 2 # 2 seconds

max_values_cool_2_0 = []
max_indexes_cool_2_0 = []
wv_abs_cool_2_0 = []
times_cool_2_0 = []

time_cool = 0

for i, txt_file in enumerate(natsorted(txt_files_2_0)):
    # Read the CSV file into a Pandas DataFrame
    df_cool = pd.read_csv(txt_file, skiprows=14, delimiter='\t', header=None)
    new_df_cool = df_cool.iloc[400:1200, ] #385 to 743 nm
    
    # Find the maximum value and its corresponding index
    #max_value_cool = new_df_cool.iloc[667,1] # 685 nm
    max_value_cool = new_df_cool[1].max()
    max_values_cool_2_0.append(max_value_cool)
    
    # Get the corresponding value from the first column
    max_index_cool = new_df_cool.loc[new_df_cool[1].idxmax(), 0]
    max_indexes_cool_2_0.append(max_index_cool)
    
    times_cool_2_0.append(time_cool)
    time_cool = time_cool + 2 # 2 seconds

max_values_cool_2_2 = []
max_indexes_cool_2_2 = []
wv_abs_cool_2_2 = []
times_cool_2_2 = []

time_cool = 0

for i, txt_file in enumerate(natsorted(txt_files_2_2)):
    # Read the CSV file into a Pandas DataFrame
    df_cool = pd.read_csv(txt_file, skiprows=14, delimiter='\t', header=None)
    new_df_cool = df_cool.iloc[400:1200, ] #385 to 743 nm
    
    # Find the maximum value and its corresponding index
    #max_value_cool = new_df_cool.iloc[667,1] # 685 nm
    max_value_cool = new_df_cool[1].max()
    max_values_cool_2_2.append(max_value_cool)
    
    # Get the corresponding value from the first column
    max_index_cool = new_df_cool.loc[new_df_cool[1].idxmax(), 0]
    max_indexes_cool_2_2.append(max_index_cool)
    
    times_cool_2_2.append(time_cool)
    time_cool = time_cool + 2 # 2 seconds

max_values_cool_2_5 = []
max_indexes_cool_2_5 = []
wv_abs_cool_2_5 = []
times_cool_2_5 = []

time_cool = 0

for i, txt_file in enumerate(natsorted(txt_files_2_5)):
    # Read the CSV file into a Pandas DataFrame
    df_cool = pd.read_csv(txt_file, skiprows=14, delimiter='\t', header=None)
    new_df_cool = df_cool.iloc[400:1200, ] #385 to 743 nm
    
    # Find the maximum value and its corresponding index
    #max_value_cool = new_df_cool.iloc[667,1] # 685 nm
    max_value_cool = new_df_cool[1].max()
    max_values_cool_2_5.append(max_value_cool)
    
    # Get the corresponding value from the first column
    max_index_cool = new_df_cool.loc[new_df_cool[1].idxmax(), 0]
    max_indexes_cool_2_5.append(max_index_cool)
    
    times_cool_2_5.append(time_cool)
    time_cool = time_cool + 2 # 2 seconds

# Plot of max. wavelength with time

fig = plt.figure(figsize=(7,3), dpi=300) # generate a figure
ax = plt.subplot(1,1,1) #(nrow, ncolumn, index) - make a subplot, there is only one

ax.plot(times_cool_1_8, max_values_cool_1_8, linewidth = 1, color = 'grey', label = '1.8A')                         
ax.plot(times_cool_2_0, max_values_cool_2_0, linewidth = 1, color = 'royalblue', label = '2.0A') 
ax.plot(times_cool_2_5, max_values_cool_2_5, linewidth = 1, color = 'firebrick', label = '2.5A') 
 
ax.set_xlim([0,3500])                   
#ax.set_ylim([600,700])                       
ax.set_xlabel(r'Time (s)')          # set axis label
ax.set_ylabel(r'Ext. @ 685 nm')              # set axis label
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(frameon = False)

plt.tight_layout()  

fig.savefig(RESULT_DIR_PATH + 'Fig_4.pdf', transparent=True)

In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import glob
from natsort import natsorted

# Assuming you have already defined DATA_UV_DIR_PATH_AGGREGATION_1_8, DATA_UV_DIR_PATH_AGGREGATION_2_0, 
# DATA_UV_DIR_PATH_AGGREGATION_2_2, and DATA_UV_DIR_PATH_AGGREGATION_2_5

# Get a list of all CSV files in each folder
txt_files_1_8 = glob.glob(os.path.join(DATA_UV_DIR_PATH_AGGREGATION_1_8, '*.txt'))
txt_files_2_0 = glob.glob(os.path.join(DATA_UV_DIR_PATH_AGGREGATION_2_0, '*.txt'))
txt_files_2_2 = glob.glob(os.path.join(DATA_UV_DIR_PATH_AGGREGATION_2_2, '*.txt'))
txt_files_2_5 = glob.glob(os.path.join(DATA_UV_DIR_PATH_AGGREGATION_2_5, '*.txt'))

# Initialize the figure and subplots
fig, axs = plt.subplots(3, 1, figsize=(7, 6), dpi=300, sharex=True)

# Function to plot cooling data
def plot_cooling_data(ax, times, max_values, label, color, ylim):
    ax.plot(times, max_values, linewidth=1, label=label, color = color)
    ax.set_ylabel('Ext. @ 685 nm')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend(frameon=False)
    ax.set_ylim(ylim)
    ax.set_xlim([0,3600])

# Plot cooling data for each subplot
plot_cooling_data(axs[0], times_cool_2_5, max_values_cool_2_5, '2.5A', 'firebrick', [0,0.35])
#plot_cooling_data(axs[1], txt_files_2_2, max_values_cool_2_2, '2.2A', 'grey', [0.05,0.35])
plot_cooling_data(axs[1], times_cool_2_0, max_values_cool_2_0, '2.0A', 'royalblue', [0.19,0.35])
plot_cooling_data(axs[2], times_cool_1_8, max_values_cool_1_8, '1.8A', 'grey', [0,0.35])

# Set common xlabel and adjust layout
plt.xlabel('Time (s)')
plt.tight_layout()

# Save or show the plot
#plt.savefig(RESULT_DIR_PATH + '2024-01-17-100auss-20aunr-7dna2--7ulbp-15c-2a.png_separated.png', transparent=True)
plt.show()

# Calculating baseline

In [ ]:
def baseline_als(y, lam, p, niter=10):
  L = len(y)
  D = sparse.csc_matrix(np.diff(np.eye(L), 2))
  w = np.ones(L)
  for i in range(niter):
    W = sparse.spdiags(w, 0, L, L)
    Z = W + lam * D.dot(D.transpose())
    z = spsolve(Z, w*y)
    w = p * (y > z) + (1-p) * (y < z)
  return z

In [ ]:
baseline_als(max_values_cool_2_0,  lam=1e7, p=0.02)

In [ ]:
fig = plt.figure(figsize=(7,3), dpi=300) # generate a figure
ax = plt.subplot(1,1,1) #(nrow, ncolumn, index) - make a subplot, there is only one

ax.plot(times_cool_2_0, (max_values_cool_2_0 - (baseline_als(max_values_cool_2_0,  lam=1.0E4, p=0.5))), linewidth = 1, color = 'firebrick')                           # plot data

ax.set_xlim([500,2000])                   
#ax.set_ylim([600,700])                       
ax.set_xlabel(r'Time (s)')          # set axis label
ax.set_ylabel(r'Ext. @ 685 nm')              # set axis label
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()  

#fig.savefig(RESULT_DIR_PATH + '2024-01-17-100auss-20aunr-7dna2--7ulbp-15c-2a.png', transparent=True)

In [ ]:
base_corr = (max_values_cool_2_0 - (baseline_als(max_values_cool_2_0,  lam=1.0E6, p=0.5)))

#base_corr[500:2000]

#print(len(base_corr[500:2000]))
print(len(base_corr))

In [ ]:
fig = plt.figure(figsize=(7,3), dpi=100) # generate a figure
ax = plt.subplot(1,1,1) #(nrow, ncolumn, index) - make a subplot, there is only one

ax.plot(base_corr[500:1500], linewidth = 1, color = 'firebrick')                           # plot data
#ax.set_title('17$^{o}$C-3.3A-35mins-Wavelength')
#ax.plot(base_corr[500:1000], linewidth = 1, color = 'firebrick')                           # plot data

#ax.set_xlim([500,2000])                   
#ax.set_ylim([600,700])                       
ax.set_xlabel(r'Time (s)')          # set axis label
ax.set_ylabel(r'Ext. @ 685 nm')              # set axis label
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()  

In [ ]:
signal = base_corr[500:1500]

# Number of sample points
N = len(signal)
# sample spacing
sampling_rate = 0.5

yf = fft(base_corr)
xf = fftfreq(N, d=1.0/sampling_rate)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from matplotlib.gridspec import GridSpec

# Assuming times_cool, max_values_cool, baseline_als, N, sampling_rate, and signal are defined

fig = plt.figure(figsize=(7, 3), dpi=600)
gs = GridSpec(1, 2)  # nrows, ncols
ax1 = fig.add_subplot(gs[0, 0])  # upper left subplot
ax2 = fig.add_subplot(gs[0, 1])  # upper right subplot

# Plot baseline-corrected data
ax1.plot(times_cool_2_0, 0.01+(max_values_cool_2_0 - baseline_als(max_values_cool_2_0, lam=1.0E4, p=0.5)), linewidth=1, color='royalblue')
ax1.set_ylim([0, 0.025])
ax1.set_xlim([500, 2000])
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Base. corr. Ext. @ 685 nm')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

# Perform Fourier transform and plot
freqs = np.fft.rfftfreq(N, d=1/sampling_rate)
fft_values = np.abs(np.fft.rfft(signal))
ax2.plot(freqs, 2*fft_values/N, linewidth=1, color='royalblue')
ax2.set_xlabel('Freq. (Hz)')
ax2.set_ylabel('Amplitude')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

# Find peaks in the Fourier transform plot
peaks, _ = find_peaks(2*fft_values/N, height=0.0003)
ax2.plot(freqs[peaks], (2*fft_values/N)[peaks], 'x', color='red', markersize=8)

# Print frequencies of the peaks
print("Frequencies of the peaks:")
for peak_index in peaks:
    print(freqs[peak_index])

plt.tight_layout()
fig.savefig(RESULT_DIR_PATH + 'Fig_4b_c.png', transparent=True)
plt.show()


In [ ]:
new_df_cool = df_cool.iloc[400:1100, ]

In [ ]:
new_df_cool.iloc[304,0]

In [ ]:
'''
Import cooling data from UV-Vis-NIR folder
'''

#================ import data =========================#

# Define the path to the folder containing the CSV files
folder_path = DATA_UV_DIR_PATH_AGGREGATION

# Get a list of all CSV files in the folder
txt_files = glob.glob(os.path.join(folder_path, '*.txt'))

print(len(txt_files))

#================ plot data heating cycle ===========================#

#fig = plt.figure(figsize=(4,2), dpi=600) # generate a figure
#ax = plt.subplot(1,1,1) #(nrow, ncolumn, index) - make a subplot, there is only one

# Initialize the maximum value and its corresponding index
max_values_cool_520 = []
max_indexes_cool_520 = []
wv_abs_cool = []
times_cool = []

time_cool = 0

for i, txt_file in enumerate(natsorted(txt_files)):
    # Read the CSV file into a Pandas DataFrame
    df_cool = pd.read_csv(txt_file, skiprows=14, delimiter='\t', header=None)
    new_df_cool = df_cool.iloc[400:1200, ] #385 to 699 nm
    
    # Find the maximum value and its corresponding index
    max_value_cool = new_df_cool.iloc[304,1] # 523 nm
    max_values_cool_520.append(max_value_cool)
    
    # Get the corresponding value from the first column
    max_index_cool_520 = new_df_cool.loc[new_df_cool.idxmax(), 0]
    max_indexes_cool_520.append(max_index_cool_520)
    
    times_cool.append(time_cool)
    time_cool = time_cool + 2 # 2 seconds
    
    # set the color

    if i == 0:
        color = 'firebrick'
        linewidth = 1
    elif i == len(txt_files)-1:
        color = 'royalblue'
        linewidth = 1
    else:
        color = 'grey'
        linewidth = 0
    
    # Plot the data
    #ax.plot(df_cool[0],df_cool[1], color = color, linewidth = linewidth)                           # plot data

#ax.set_xlim([350,800])                    # set wavelength limit
#ax.set_ylim([-0.1,0.8])                       # set absorbance limit
#ax.set_xlabel(r'Wavelength (nm)')          # set axis label
#ax.set_ylabel(r'Extinction ')              # set axis label
#ax.axvline(523, color='firebrick')

#plt.tight_layout()  


# Plot of max. wavelength with time

fig = plt.figure(figsize=(7,3), dpi=300) # generate a figure
ax = plt.subplot(1,1,1) #(nrow, ncolumn, index) - make a subplot, there is only one

ax.plot(times_cool, max_values_cool_520, linewidth = 1, color = 'firebrick', label = '523 nm')                           # plot data
ax.plot(times_cool, max_values_cool_2_0, linewidth = 1, color = 'royalblue', label = '685 nm')                           # plot data
#ax.set_title('17$^{o}$C-3.3A-35mins-Wavelength')

ax.set_xlim([0,3500])                        
#ax.set_ylim([0.05,0.2])                       
ax.set_xlabel(r'Time(s)')          # set axis label
ax.set_ylabel(r'Extinction')              # set axis label
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(frameon = False)

plt.tight_layout()  
plt.savefig(RESULT_DIR_PATH_SI + 'Fig_S9.png', transparent = True, bbox_inches = 'tight')


fig = plt.figure(figsize=(7,3), dpi=300) # generate a figure
ax = plt.subplot(1,1,1) #(nrow, ncolumn, index) - make a subplot, there is only one

ax.plot(times_cool, max_indexes_cool_520, linewidth = 1, color = 'royalblue')                           # plot data
#ax.set_title('17$^{o}$C-3.3A-35mins-Wavelength')

ax.set_xlim([0,3500])                        
ax.set_ylim([680,700])                       
ax.set_xlabel(r'Time(s)')          # set axis label
ax.set_ylabel(r'$\lambda_{max.}$ (nm)')              # set axis label
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()  
plt.savefig(RESULT_DIR_PATH_SI + 'Fig_S10.png', transparent = True, bbox_inches = 'tight')